## Exercici 1
Realitza web scraping d'una pàgina de la borsa de Madrid (https://www.bolsamadrid.es) utilitzant BeautifulSoup i Selenium.

**BeautifulSoup**

En primer lugar, inspecciono el contenido de la página web

In [ ]:
import requests

URL = "https://www.bolsamadrid.es/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000&punto=indice"
page = requests.get(URL)

print(page.text)

Decido la información que quiero obtener, el conjunto de diferencias de precios de las empresas del IBEX35

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(page.content, "html.parser")

Localizo las clases HTML que me interesan y recojo la información. Aprovecho para comprobar que todos los datos que necesito están presentes y que no hay datos extra

In [ ]:
empresas = soup.find_all("td", class_="DifFlBj")
difs = soup.find_all("td", class_="DifClBj")

empresas_bj = []
for empresa in empresas:
    empresas_bj.append(empresa.text.strip())

print(len(empresas_bj), empresas_bj)

difs_bj = []
for dif in difs:
    difs_bj.append(dif.text.strip())
    
print(len(difs_bj), difs_bj)

In [ ]:
empresas = soup.find_all("td", class_="DifFlSb")
difs = soup.find_all("td", class_="DifClSb")

empresas_sb = []
for empresa in empresas:
    empresas_sb.append(empresa.text.strip())

print(len(empresas_sb), empresas_sb)

difs_sb = []
for dif in difs:
    difs_sb.append(dif.text.strip())
    
print(len(difs_sb), difs_sb)

Compruebo que, junto a la información relativa a las empresas, se incluye información del IBEX35 en conjunto (el nombre, IBEX35 y dos precios relativos a la media anual y la de ese momento).

Por tanto, elimino de la colección de datos la información general del IBEX35. Primero el nombre:

In [ ]:
# Averiguar si el IBEX ha subido o ha bajado
if "IBEX 35®" in empresas_sb:
    print(empresas_sb)
    index = empresas_sb.index("IBEX 35®")
    empresas_sb.remove("IBEX 35®")
    print(empresas_sb)
elif "IBEX 35®" in empresas_bj:
    print(empresas_bj)
    index = empresas_bj.index("IBEX 35®")
    empresas_bj.remove("IBEX 35®")
    print(empresas_bj)

 Y luego sus dos precios, tanto si indican incremento como disminución

In [ ]:
print(len(empresas_sb), len(difs_sb))

while len(empresas_sb) < len(difs_sb):
    difs_sb.remove(f"{difs_sb[0]}")
    
print(len(empresas_sb), len(difs_sb))

In [ ]:
print(len(empresas_bj), len(difs_bj))

while len(empresas_bj) < len(difs_bj):
    index= difs_bj[0]
    difs_bj.remove(f"{difs_bj[0]}")
    
print(len(empresas_bj), len(difs_bj))

De esta manera, las longitudes de todas las listas de datos (empresas y precios) cuadran y puedo estudiar los datos y trabajar con ellos. Primero creo un dataframe:

In [ ]:
import pandas as pd

df_ibex = pd.DataFrame()

df_ibex["Empresa"] = empresas_bj + empresas_sb
df_ibex["Dif"] = difs_bj + difs_sb
df_ibex = df_ibex.sort_values(by=['Empresa'])

display(df_ibex)

In [ ]:
df_ibex.info()

Convierto la columna Dif, los precios, a float

In [ ]:
df_ibex['Dif'] = df_ibex['Dif'].str.replace(',','.')
df_ibex['Dif'] = pd.to_numeric(df_ibex['Dif'])
df_ibex.info()

Y visualizo los datos en un gráfico:

In [ ]:
from matplotlib import pyplot as plt
import seaborn

a4_dims = (11.7, 8.27)
fig, ax = plt.subplots(figsize=a4_dims)
seaborn.barplot(x="Dif", y="Empresa", data=df_ibex)
plt.show()

**Selenium**

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from collections import namedtuple
from os.path import isfile
from threading import Thread
import csv
from time import sleep
from datetime import datetime


BOLSA_PRECIOSIBEX ='https://www.bolsamadrid.es/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000&punto=indice'
precios_sesion = namedtuple('precios_sesion', ['company', 'dif', 'timestamp'])


class PreciosIbex():
    def __init__(self):

        options = webdriver.ChromeOptions()
        options.add_argument("--start-maximized")
        options.add_argument('--log-level=3')
        options.add_argument('--headless')

        self.driver = webdriver.Chrome(
            executable_path="chromedriver",
            options=options
        )
        self.driver.get(BOLSA_PRECIOSIBEX)

        self.empresas_sb = []
        self.empresas_bj = []
        self.difs_sb = []
        self.difs_bj = []
        
        self.empresas_all = []
        self.difs_all = []
        self.horas = []
        
        self.df_ibex = pd.DataFrame()
        
        self.datos()
                
        self.fecha = datetime.date(datetime.now())
        self.database_path= f"precios_IBEX35_{self.fecha}.csv"
        self.database = []
        
        if isfile(self.database_path):
            with open(self.database_path, newline='') as dbfile:
                dbreader = csv.reader(dbfile)
                next(dbreader)
        
        self.thread = Thread(target=self.maintain)
        self.thread.daemon = True 
        self.thread.start()
        
    
    def maintain(self):
        while True:
            self.datos()
            self.update_db()
            sleep(20)
    
    
    def save_db(self):
        with open(self.database_path,'w', newline='') as dbfile:
            dbwriter = csv.writer(dbfile)
            dbwriter.writerow(list(precios_sesion._fields))
            for entry in self.database:
                dbwriter.writerow(list(entry))
                

    def update_db(self):
        check = ((
            (len(self.empresas_sb) > 0 and len(self.difs_sb) > 0) 
            or 
            (len(self.empresas_bj) > 0 and len(self.difs_bj) > 0)
        ))

        if check:
            for i in range(len(self.empresas_all)):
                self.database.append([self.empresas_all[i], self.difs_all[i], self.horas[i]])
                self.save_db()

        
    def datos(self):
        '''
        Recabar información de la página relativa a cambios en el IBEX
        '''
        self.driver.get(BOLSA_PRECIOSIBEX)
        empresas_sb = self.driver.find_elements(By.CLASS_NAME,'DifFlSb')
        empresas_bj = self.driver.find_elements(By.CLASS_NAME,'DifFlBj')
        difs_sb = self.driver.find_elements(By.CLASS_NAME,'DifClSb')
        difs_bj = self.driver.find_elements(By.CLASS_NAME,'DifClBj')
        horas = self.driver.find_elements(By.CLASS_NAME,'Ult')
        
        self.empresas_sb = [i.text for i in empresas_sb]
        self.empresas_bj = [i.text for i in empresas_bj]
        self.difs_sb = [i.text for i in difs_sb]
        self.difs_bj = [i.text for i in difs_bj]
        self.horas = [i.text for i in horas]
        
        self.eliminar_info_gral_IBEX()
        print(self.horas)
        self.convertir_float()
        self.crear_tabla()
    
    
    def eliminar_info_gral_IBEX(self):
              
        if "IBEX 35®" in self.empresas_sb:
            self.empresas_sb.remove("IBEX 35®")
        elif "IBEX 35®" in self.empresas_bj:
            self.empresas_bj.remove("IBEX 35®")

        while len(self.empresas_sb) < len(self.difs_sb):
            self.difs_sb.remove(f"{self.difs_sb[0]}")
        while len(self.empresas_bj) < len(self.difs_bj):
            self.difs_bj.remove(f"{self.difs_bj[0]}")
        
        self.empresas_all = self.empresas_sb + self.empresas_bj        
        self.difs_all = self.difs_sb + self.difs_bj
        
        while len(self.difs_all) < len(self.horas):
            self.horas.remove(f"{self.horas[0]}")
        
        
    def convertir_float(self):
        floated_difs = []
        for dif in self.difs_all:
            dif = dif.replace(",", ".")
            dif = float(dif)
            floated_difs.append(dif)
        self.difs_all = floated_difs
        
        
    def crear_tabla(self):
        self.df_ibex["Empresa"] = self.empresas_bj + self.empresas_sb
        self.df_ibex["Dif"] = self.difs_bj + self.difs_sb
        self.df_ibex['Dif'] = self.df_ibex['Dif'].str.replace(',','.')
        self.df_ibex['Dif'] = pd.to_numeric(self.df_ibex['Dif'])
        self.df_ibex['Hora'] = self.horas
        
        
    def ver_grafico(self):
        self.crear_tabla()
        a4_dims = (11.7, 8.27)
        fig, ax = plt.subplots(figsize=a4_dims)
        seaborn.barplot(x="Dif", y="Empresa", data=df_ibex)
        
    
    def ver_datos(self):
        self.crear_tabla()
        display(self.df_ibex)
    

In [ ]:
precios_ibex = PreciosIbex()

In [ ]:
precios_ibex.ver_grafico()

In [ ]:
precios_ibex.ver_datos()

**Scrapy**

In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
import json

"""
import sys
from twisted.internet import default

if 'twisted.internet.reactor' in sys.modules:
    del sys.modules['twisted.internet.reactor']

default.install()
"""

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('birds_results.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [2]:
import logging
from string import ascii_uppercase


class BirdSpider(scrapy.Spider):
    name = "birds"
    start_urls = [
        f'https://seo.org/listado-aves-2/?letra={letter}' for letter in ascii_uppercase
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEEDS':{'birds_results.csv':{'format':'csv'}}
    }
    
    def parse(self, response):
        for bird in response.css('div.contenedor_txt'):
            yield {
                'namme_common': bird.css('h3 a::text').extract_first(),
                'name_scientific': bird.css('p::text').extract_first(),
            }

In [3]:
import sys
from twisted.internet import default

# Permite ejecutar este mismo bloque de código sin que genere "reactoralreadyinstalled.error"
if 'twisted.internet.reactor' in sys.modules:
    del sys.modules['twisted.internet.reactor']

    
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(BirdSpider)
process.start()

2022-04-04 19:43:57 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-04-04 19:43:57 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.2.0, Python 3.9.7 (default, Sep 16 2021, 08:50:36) - [Clang 10.0.0 ], pyOpenSSL 21.0.0 (OpenSSL 1.1.1l  24 Aug 2021), cryptography 3.4.8, Platform macOS-10.16-x86_64-i386-64bit
2022-04-04 19:43:57 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 30,
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
